# Binary evolution between two non-degenerate stars

In this notebook we show the features present in the evolution of two non-degenerate, rotating stars

These stars are aimed at being possible progenitors of the high-mass X-ray binary (HMXB) J08408-...

The initial conditions for the binary are that of a circular orbit, with the stars syncronized to the orbital period. The metallicity is assumed to be solar.

### Config stuff

In [ ]:
save_figures = False

In [ ]:
%load_ext autoreload
%autoreload 2

# ---------------------------------------------
# Libraries used
import os
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FixedLocator, MultipleLocator
from matplotlib import colors
import pandas as pd
from scipy.optimize import bisect

from mesa_reader import MesaInfo
import kipplotter.mkipp as mkipp

# ---------------------------------------------
# Set some global options
np.seterr(all='ignore')

# ---------------------------------------------
# Load data
root = '..'
run_location = f'{root}/data/raw/star+star-2/binary_to_postRLOF'
config_location = f'{root}/config'

star1 = MesaInfo(f'{run_location}/LOGS1/history.data')
star2 = MesaInfo(f'{run_location}/LOGS2/history.data')
binary = MesaInfo(f'{run_location}/binary_history.data')

mpl_style = f'{config_location}/style.mpl'
paper_style = f'{config_location}/paper-style.mpl'

# plot config
MTcases  = ['Case A',  'Case B' ]
MTcolors = ['#DEBA93', '#D9B6C4']
starcolors = ['#2D3C52', '#408191']
mix_zones = ['convection', 'semiconvection', 'thermohaline', 'overshooting']
mix_colors = ['Chartreuse', 'red', 'Gold', 'purple']
mix_hatches = ['///', '\\\\\\\\\\\\', '||', 'xxx']
star1color = '#408191'  # '#D9B6C4'
star2color = '#3F4364'

### Utility functions

In [ ]:
def group_consecutives(vals, step=1):
    '''Return list of consecutive lists of numbers from vals (number list)
    '''
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(int(v))
        else:
            run = [int(v)]
            result.append(run)
        expect = int(v) + step
    return result

def find_tams(center_h1, model):
    for m in model:
        if center_h1[int(m)] < 1e-5:
            return int(m)

---

# Hertzsprung-Russell diagram

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(3.4,3.4), nrows=1, ncols=1)
left, bottom, width, height = [0.65, 0.72, 0.14, 0.14]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
ax.set_xlabel('$\\log\,T_{\\rm eff}/{\\rm K}$')
ax.set_ylabel('$\\log\,L/L_\\odot$');

# constant radii lines
x = np.linspace(0, 8, 100)
for r in (1, 10, 100):
    y = 4*x + 2*np.log10(r) - 15.04680947
    ax.plot(x, y, linestyle=':', color='gray', alpha=0.6, zorder=-99)
ax.annotate('$1 \\; R_\\odot$', xy=(4.96, 4.73), color='gray', rotation=290)
ax.annotate('$10 \\; R_\\odot$', xy=(4.465, 4.73), color='gray', rotation=290);

# ZAMS line for Z=0.02
# _, lt, ll = np.loadtxt(fname='zams_hr.data', unpack=True)
# ax.plot(lt, ll, linestyle='--', lw=1, color='black', zorder=-99)

# primary
logTeff = star1.data.log_Teff
logL = star1.data.log_L
pcenterH1 = star1.data.center_h1   # we have pcenter because its needed to match MT phases for secondary HR
pcenterHe4 = star1.data.center_he4
modnum = star1.data.model_number
nm = find_tams(pcenterH1, modnum)

ax.plot(logTeff[100:], logL[100:], color=star1color, label='primary (25 M$_\\odot$)')
ax2.plot(logTeff[100:], logL[100:], color=star1color)

# shade with gray regions of MT phases
mask = (binary.data.rl_relative_overflow_1 > 0)
model_numbers = group_consecutives(binary.data.model_number[mask])
for i in range(len(model_numbers)):
    h1 = pcenterH1[int(model_numbers[i][0])]
    he4 = pcenterHe4[int(model_numbers[i][0])]
    if h1 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[0], lw=6, zorder=-98)
    elif he4 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[1], lw=6, zorder=-98)
    else:
        print('Case C not expected')

# secondary
logTeff = star2.data.log_Teff
logL = star2.data.log_L
modnum = star2.data.model_number

# ax.plot(logTeff[100:-2000], logL[100:-2000], color='orange', label='secondary (22 M$_\\odot$)')
ax.plot(logTeff[100:], logL[100:], color=star2color, label='secondary (22 M$_\\odot$)')
ax2.plot(logTeff[100:], logL[100:], color=star2color)

mask = (binary.data.rl_relative_overflow_1 > 0)
model_numbers = group_consecutives(binary.data.model_number[mask])
for i in range(len(model_numbers)):
    h1 = pcenterH1[int(model_numbers[i][0])]
    he4 = pcenterHe4[int(model_numbers[i][0])]
    if h1 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[0], lw=6, zorder=-98)
    elif he4 > 1e-6:
        # numbers = model_numbers[i][:-1400]
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[1], lw=6, zorder=-98)
    else:
        print('Case C not expected')

# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0, alpha=1)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
fig.legend(handles, legends,
           handlelength=1.8,
           title='MT phases',
           title_fontsize=9,
           fancybox=True, frameon=True, facecolor='lightgray', edgecolor='dimgray', framealpha=1.0,
           loc='center', ncol=1, bbox_to_anchor=(0.18,0.15), bbox_transform=ax.transAxes)

ax.legend(loc='upper center', ncol=1, columnspacing=0.6, bbox_to_anchor=(0.27,0.96));

# axis limits & ticks
ax.set_xticks(np.arange(4.3, 5.4,0.2))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_xlim(5.25, 4.35)
ax.set_yticks(np.arange(4.6, 6.2, 0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_ylim(4.7, 5.9)

ax2.set_xticks(np.arange(4.4, 5, 0.05))
ax2.xaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_xlim(4.575, 4.475)

ax2.set_yticks(np.arange(5.4, 6, 0.05))
ax2.yaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_ylim(5.42, 5.53)

# reduce fontsize for inset plot
for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(6);
    
# initial orbital period
# ax.annotate('$P_{\\rm orb} = 3$ days', xy=(4.65, 5.72),
#             bbox=dict(facecolor='lightgray', edgecolor='dimgray', boxstyle='round, pad=0.4'));


plt.subplots_adjust(left=0.15);
# if save_figures: plt.savefig('hr.png', dpi=250);

#### brief check of model number in secondary HR diagram

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(2.4,2.4), nrows=1, ncols=1)

# axis labels
ax.set_xlabel('$\\log\,T_{\\rm eff}/{\\rm K}$')
ax.set_ylabel('$\\log\,L/L_\\odot$');

# secondary
logTeff = star2.data.log_Teff
logL = star2.data.log_L
modnum = star2.data.model_number

ni = 100
nf = 10850

# ax.plot(logTeff[100:-2000], logL[100:-2000], color='orange', label='secondary (22 M$_\\odot$)')
cb = ax.scatter(logTeff[ni:nf], logL[ni:nf], c=modnum[ni:nf], s=0.5);

plt.colorbar(cb)

# axis limits
ax.set_xlim(4.6, 4.4)
ax.set_ylim(5.35, 5.53);

# Timescale for each mass-transfer case

In [ ]:
mask = bbinary.data.rl_relative_overflow_1 > 0e0
numbers = group_consecutives(bbinary.data.model_number[mask])

time_A = bbinary.data.age[numbers[0]]
time_B = bbinary.data.age[numbers[1]]
model_number_A = bbinary.data.model_number[numbers[0]]
model_number_B = bbinary.data.model_number[numbers[1]]

print('- Case A:')
print(f'\tt0: {time_A[0]:.5e}, t1: {time_A[-1]:.5e}, dt: {time_A[-1]-time_A[0]:.5e}')
print(f'\tmod_t0: {int(model_number_A[0])}, mod_t1: {int(model_number_A[-1])}')
print('\n- Case B:')
print(f'\tt0: {time_B[0]:.5e}, t1: {time_B[-1]:.5e}, dt: {time_B[-1]-time_B[0]:.5e}')
print(f'\tmod_t0: {int(model_number_B[0])}, mod_t1: {int(model_number_B[-1])}')

# Mass transfer & mass accretion

In [ ]:
plt.style.use('../config/style.mpl')
fig, axs = plt.subplots(figsize=(4.2,4.2), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)
axs[0].set_ylim([-0.05, 1.05]); axs[0].set_yticks(np.arange(0.0, 1.2, 0.2))
axs[1].set_ylim([-8.5, -1.5]); axs[1].set_yticks(np.arange(-8,-1))
axs[1].set_xticks(np.arange(8, 25, 2)); axs[1].set_xlim([9.2, 24.5])
axs[0].set_ylabel('$f_{\\rm mt} = - \\dot{M}_2 / \\dot{M}_1$')
axs[1].set_ylabel('$\\log\,\\mid\\dot{M}\\mid$ [M$_\\odot$ yr$^{-1}$]')
axs[1].set_xlabel('$M_1$ [M$_\\odot$]')

mask = bbinary.data.rl_relative_overflow_1 > 0e0
numbers = group_consecutives(bbinary.data.model_number[mask])

mass_1A = bbinary.data.star_1_mass[numbers[0]]
mdot_1A = np.power(10, bbinary.data.lg_mstar_dot_1[numbers[0]])
mdot_2A = np.power(10, bbinary.data.lg_mstar_dot_2[numbers[0]])
mass_1B = bbinary.data.star_1_mass[numbers[1]]
mdot_1B = np.power(10, bbinary.data.lg_mstar_dot_1[numbers[1]])
mdot_2B = np.power(10, bbinary.data.lg_mstar_dot_2[numbers[1]])

axs[0].plot(mass_1A, mdot_2A/mdot_1A, color='#611D52')
axs[0].plot(mass_1A, mdot_2A/mdot_1A, color=MTcolors[0], lw=6, zorder=-1)
axs[0].plot(mass_1B, mdot_2B/mdot_1B, color='#611D52')
axs[0].plot(mass_1B, mdot_2B/mdot_1B, color=MTcolors[1], lw=6, zorder=-1)

axs[1].plot(mass_1A, np.log10(mdot_1A), color=starcolors[0])
axs[1].plot(mass_1B, np.log10(mdot_1B), color=starcolors[0], label='$\\dot{M}_1$')
axs[1].plot(mass_1A, np.log10(mdot_2A), color=starcolors[1])
axs[1].plot(mass_1B, np.log10(mdot_2B), color=starcolors[1], label='$\\dot{M}_2$')

# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
axs[0].legend(handles, legends,
              handlelength=1.8,
              title='MT phases', title_fontsize=9,
              fancybox=True, frameon=True,
              facecolor='lightgray', edgecolor='dimgray', framealpha=1.0,
              loc='center', ncol=1,
              bbox_to_anchor=(0.8,0.4), bbox_transform=axs[0].transAxes);

axs[1].legend(loc='center', bbox_to_anchor=(0.8,0.4), bbox_transform=axs[1].transAxes);

At around 18 M$_\odot$ and during the Case A of mass-transfer (MT), the accretor stops to accrete all the material
that reaches its surface, that is why we see a drop in the accretion efficiency ($f_{\rm mt}$)

This same sudden drop in $f_{\rm mt}$ is obtained for the Case B of MT, around 11 M$_\odot$ into the donor star.
In this case, the value of $f_{\rm mt}$ is pretty noisy perhaps due to numerical issues in this faster MT episode

# Rotation rate of accretor

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(3.4,3.4), nrows=1, ncols=1)
ax.set_xticks(np.arange(8, 25, 2)); axs[1].set_xlim([9.2, 24.5])
ax.set_xlabel('$M_1$ [M$_\\odot$]')
ax.set_ylabel('$\\Omega / \\Omega_{\\rm crit}$')

mask = bbinary.data.rl_relative_overflow_1 > 0e0
numbers = group_consecutives(bbinary.data.model_number[mask])

mass_1A = bbinary.data.star_1_mass[numbers[0]]
omega_div_omega_crit_1A = sstar.data.surf_avg_omega_div_omega_crit[numbers[0]]
mass_1B = bbinary.data.star_1_mass[numbers[1]]
omega_div_omega_crit_1B = sstar.data.surf_avg_omega_div_omega_crit[numbers[1]]


ax.plot(mass_1A, omega_div_omega_crit_1A, color=starcolors[1])
ax.plot(mass_1A, omega_div_omega_crit_1A, color=MTcolors[0], lw=6, zorder=-1)
ax.plot(mass_1B, omega_div_omega_crit_1B, color=starcolors[1])
ax.plot(mass_1B, omega_div_omega_crit_1B, color=MTcolors[1], lw=6, zorder=-1)

# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
ax.legend(handles, legends,
          handlelength=1.8,
          title='MT phases', title_fontsize=9,
          fancybox=True, frameon=True,
          facecolor='lightgray', edgecolor='dimgray', framealpha=1.0,
          loc='center', ncol=1,
          bbox_to_anchor=(0.8,0.5), bbox_transform=ax.transAxes);

# inset plot
left, bottom, width, height = [0.35, 0.64, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.set_xlim([9.65, 9.95])
ax2.set_ylim([0.76, 0.96])

ax2.plot(mass_1B, omega_div_omega_crit_1B, color=starcolors[1], lw=0.5)
ax2.plot(mass_1B, omega_div_omega_crit_1B, color=MTcolors[1], lw=2, zorder=-1)

# reduce fontsize for inset plot
for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(6);

# Luminosity evolution during MT

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(3.4,3.4), nrows=1, ncols=1)
ax.set_xticks(np.arange(8, 25, 2)); axs[1].set_xlim([9.2, 24.5])
ax.set_xlabel('$M_1$ [M$_\\odot$]')
ax.set_ylabel('$\\log\\,(L/L_\\odot)$')

mask = bbinary.data.rl_relative_overflow_1 > 0e0
numbers = group_consecutives(bbinary.data.model_number[mask])

mass_1A = bbinary.data.star_1_mass[numbers[0]]
log_L_2A = sstar.data.log_L[numbers[0]]
log_Lnuc_2A = sstar.data.log_Lnuc[numbers[0]]
mass_1B = bbinary.data.star_1_mass[numbers[1]]
log_L_2B = sstar.data.log_L[numbers[1]]
log_Lnuc_2B = sstar.data.log_Lnuc[numbers[1]]

ax.plot(mass_1A, log_L_2A, color=starcolors[1])
ax.plot(mass_1A, log_Lnuc_2A, color='black', ls='--')
ax.plot(mass_1B, log_L_2B, color=starcolors[1])
ax.plot(mass_1B, log_Lnuc_2B, color='black', ls='--');

## Case A of MT: hook to higher L in donor evolution

In [ ]:
log_L_1A = pstar.data.log_L[numbers[0]]
amin = np.argmin(log_L_1A)
model_number_of_hook = int(bbinary.data.model_number[int(numbers[0][amin])])
print(f'model number @ change in slope: {model_number_of_hook}')

### Kippenhahn diagram near hook

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(4,3))
ax.set_xlabel('model number')
ax.set_ylabel('$m/M_\\odot$')
ax.set_xlim([300, model_number_A[-1]])

args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS1'], history_paths=[f'{run_location}/LOGS1/history.data'],
    xaxis='model_number', time_units='Myr', yaxis='mass', identifier='eps_nuc',
    function_on_xaxis = lambda x: x, #np.log10(max_age+0.01 - x),
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=True,
    levels=np.arange(0,6.1,0.1), decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax, ticks=np.arange(0,7,1))
bar.set_label('$\\log\\,\\epsilon_{\\rm nuc}$')

ax.axvline(x=2622, ls=':', color='black')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))
# legends.extend(('He core', 'C core', 'O core'))
# handles.extend((f('blue'), f('red'), f('green')))
fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

### Mass fractions near hook

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(4,3))
ax.set_yscale('log')
ax.set_xlabel('model number')
ax.set_ylabel('mass fraction')

ax.set_xlim([2000, 3000])
ax.set_ylim([1e-4, 1])

ax.plot(pstar.data.model_number, pstar.data.surface_h1)
ax.plot(pstar.data.model_number, pstar.data.surface_he4)
ax.plot(pstar.data.model_number, pstar.data.surface_c12)
ax.plot(pstar.data.model_number, pstar.data.surface_n14)

ax.axvline(x=2622, ls=':', color='black');

# Kippenhahn plot for the accretor

## During Case A of MT

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(4,3))
ax.set_xlabel('$t$ [Myr]')
ax.set_ylabel('$m/M_\\odot$')
ax.set_xlim([time_A[0]/1e6, time_A[-1]/1e6])

args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='Myr', yaxis='mass', identifier='eps_nuc',
    function_on_xaxis = lambda x: x, #np.log10(max_age+0.01 - x),
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=True,
    levels=np.arange(0,6.1,0.1), decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax, ticks=np.arange(0,7,1))
bar.set_label('$\\log\\,\\epsilon_{\\rm nuc}$')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))
# legends.extend(('He core', 'C core', 'O core'))
# handles.extend((f('blue'), f('red'), f('green')))
fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

In [ ]:
profile_acc_init_A = MesaInfo(f'{run_location}/LOGS2/profile7.data')
profile_acc_end_A = MesaInfo(f'{run_location}/LOGS2/profile47.data')

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(3.2,3.2))
ax.set_yscale('log')

ax.plot(profile_acc_init_A.data.mass, profile_acc_init_A.data.h1, color='C0')
ax.plot(profile_acc_end_A.data.mass, profile_acc_end_A.data.h1, color='C0', ls='--')

ax.plot(profile_acc_init_A.data.mass, profile_acc_init_A.data.he4, color='C1')
ax.plot(profile_acc_end_A.data.mass, profile_acc_end_A.data.he4, color='C1', ls='--')

ax.plot(profile_acc_init_A.data.mass, profile_acc_init_A.data.n14, color='C2')
ax.plot(profile_acc_end_A.data.mass, profile_acc_end_A.data.n14, color='C2', ls='--')

## During Case B of MT

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(4,3))
ax.set_xlabel('$t$ [Myr]')
ax.set_ylabel('$m/M_\\odot$')
ax.set_xlim([time_B[0]/1e6, time_B[-1]/1e6])

args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='Myr', yaxis='mass', identifier='eps_nuc',
    function_on_xaxis = lambda x: x, #np.log10(max_age+0.01 - x),
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=True,
    levels=np.arange(0,6.1,0.1), decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax, ticks=np.arange(0,7,1))
bar.set_label('$\\log\\,\\epsilon_{\\rm nuc}$')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))
# legends.extend(('He core', 'C core', 'O core'))
# handles.extend((f('blue'), f('red'), f('green')))
fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(4,3))
ax.set_xlabel('$t$ [Myr]')
ax.set_ylabel('$m/M_\\odot$')
ax.set_xlim([time_B[0]/1e6, time_B[-1]/1e6])

args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='Myr', yaxis='mass', identifier='he4',
    function_on_xaxis = lambda x: x,
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=False,
    levels=np.arange(0,1.1,0.2),
    decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax, ticks=np.arange(0,7,1))
bar.set_label('$\\Omega / \\Omega_{\\rm crit}$');

In [ ]:
profile_acc_init_B = MesaInfo(f'{run_location}/LOGS2/profile51.data')
profile_acc_end_B = MesaInfo(f'{run_location}/LOGS2/profile140.data')

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(3.2,3.2))

ax.plot(profile_acc_init_B.data.mass, profile_acc_init_B.data.omega_div_omega_crit, color='C0')
ax.plot(profile_acc_end_B.data.mass, profile_acc_end_B.data.omega_div_omega_crit, color='C1')

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(3.2,3.2))
ax.set_yscale('log')

ax.plot(profile_acc_init_B.data.mass, profile_acc_init_B.data.h1, color='C0')
ax.plot(profile_acc_end_B.data.mass, profile_acc_end_B.data.h1, color='C0', ls='--')

ax.plot(profile_acc_init_B.data.mass, profile_acc_init_B.data.he4, color='C1')
ax.plot(profile_acc_end_B.data.mass, profile_acc_end_B.data.he4, color='C1', ls='--')

ax.plot(profile_acc_init_B.data.mass, profile_acc_init_B.data.n14, color='C2')
ax.plot(profile_acc_end_B.data.mass, profile_acc_end_B.data.n14, color='C2', ls='--')